In [ ]:
import csv
import copy
import argparse
import itertools
from collections import Counter
from collections import deque
import pandas as pd
import cv2
import numpy as np
import mediapipe as mp
import time
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# timer.py


class TimerError(Exception):
    """A custom exception used to report errors in use of Timer class"""

class Timer:
    def __init__(self):
        self._start_time = None

    def start(self):
        """Start a new timer"""
        if self._start_time is not None:
            raise TimerError(f"Timer is running. Use .stop() to stop it")

        self._start_time = time.perf_counter()

    def show(self):
        """Stop the timer, and report the elapsed time"""
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")

        elapsed_time = time.perf_counter() - self._start_time
        return elapsed_time
    def stop(self):
        self._start_time=None

# Function to calculate landmark list
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Iterate over each landmark point
    for _, landmark in enumerate(landmarks.landmark):
        # Convert normalized coordinates to pixel coordinates
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        # landmark_z = landmark.z  # Uncomment this line if you need the z-coordinate

        # Append the pixel coordinates to the landmark_point list
        landmark_point.append([landmark_x, landmark_y])

    return landmark_point


# Function to preprocess landmarks
def pre_process_landmark(landmark_list):
    # Implement your preprocessing logic here
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list
    pass

# Initialize hands module


# OpenCV VideoCapture
while True:
    mode=int(input("Enter 1 for loading 2 for training 3 for predicting"))
    if mode==3:
        mp_hands = mp.solutions.hands
        mp_drawing = mp.solutions.drawing_utils
        hands = mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5)
        cap = cv2.VideoCapture(0)  # Use the appropriate index or video file path

        while cap.isOpened():
            ret, frame = cap.read()

            # BGR 2 RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Flip on horizontal
            image = cv2.flip(image, 1)
            debug_image = copy.deepcopy(image)

            # Set flag
            image.flags.writeable = False

            # Detections
            results = hands.process(image)

            # Set flag to true
            image.flags.writeable = True

            # RGB 2 BGR
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            with open("label.txt",'r') as f:
                label=f.read().split()
            # Rendering results
            if results.multi_hand_landmarks:
                for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                    landmark_list = calc_landmark_list(debug_image, hand_landmarks)
                    pre_processed_landmark_list = pre_process_landmark(landmark_list)
                    x, y, w, h = cv2.boundingRect(np.array(landmark_list))
                    cv2.rectangle(debug_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    hand_label = "Left Hand" if handedness.classification[0].label == "Left" else "Right Hand"
                    cv2.putText(debug_image, hand_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                    mp_drawing.draw_landmarks(debug_image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                                                  mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                                  mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                                  )
                data2 = pd.DataFrame([pre_processed_landmark_list])
                xi = data2.iloc[:,:]
                pred = knn.predict(xi)
                # acc=knn.score()
                op=label[int(pred)]
                print(op)
                time.sleep(1)
                cv2.putText(debug_image,str(op), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 255, 255), 2)
                time.sleep(1)
                cv2.imshow('Hand Tracking', debug_image)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

        cap.release()
        cv2.destroyAllWindows()
        hands.close()
    elif mode==2:
        data=pd.read_csv('training_data.csv',header=None)
        y=data.iloc[:,0]
        x=data.iloc[:,1:]
        xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.25,random_state=0)
        knn = KNeighborsClassifier(n_neighbors=3)
        knn.fit(xtrain,ytrain)
        pred=knn.predict(xtest)
        acc=knn.score(xtest,ytest)
        print(pred,acc*100," %")
    elif mode==1:
        mp_hands = mp.solutions.hands
        mp_drawing = mp.solutions.drawing_utils
        hands = mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5)
        cap = cv2.VideoCapture(0)  # Use the appropriate index or video file path
        keyy = 1
        t = Timer()
        while keyy != 0:
            label = input("Enter label")
            label=" "+label
            with open("label.txt",'r') as f:
                la=f.read().split()
                size=len(la)
                print(size)
            with open('label.txt','a+') as f:
                print(f.read())
                f.write(label)
            t.start()
            while t.show() < 120.0 and cap.isOpened():
                ret, frame = cap.read()

                # BGR 2 RGB
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                # Flip on horizontal
                image = cv2.flip(image, 1)
                debug_image = copy.deepcopy(image)

                # Set flag
                image.flags.writeable = False

                # Detections
                results = hands.process(image)

                # Set flag to true
                image.flags.writeable = True

                # RGB 2 BGR
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                # Rendering results
                if results.multi_hand_landmarks:
                    for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                        landmark_list = calc_landmark_list(debug_image, hand_landmarks)
                        l=pre_process_landmark(landmark_list)
                        l.insert(0,size)
                        #print("Iterations:",l)
                        #print(pre_processed_landmark_list)
                        x, y, w, h = cv2.boundingRect(np.array(landmark_list))
                        cv2.rectangle(debug_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                        hand_label = "Left Hand" if handedness.classification[0].label == "Left" else "Right Hand"
                        cv2.putText(debug_image, hand_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                        mp_drawing.draw_landmarks(debug_image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                                                  mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                                  mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                                  )
                        with open("training_data.csv",'a+',newline='') as f:
                            writer=csv.writer(f)
                            #print(l)
                            writer.writerow(l)   
            keyy = int(input("Continue?"))
            t.stop()

        cap.release()
        cv2.destroyAllWindows()
        hands.close()
    else:
        break

                



# Release the VideoCapture and hands objects



Enter 1 for loading 2 for training 3 for predicting2
[22  2 28 ... 22  1  2] 98.905570366213  %
Enter 1 for loading 2 for training 3 for predicting1
Enter labelalphabet_L
29

Continue?0
Enter 1 for loading 2 for training 3 for predicting2
[20 24 22 ... 10  1 16] 98.96222498962224  %
Enter 1 for loading 2 for training 3 for predicting1
Enter labelalphabet_H
30

Continue?0
Enter 1 for loading 2 for training 3 for predicting2
[22  1 17 ...  1 22  0] 98.66935483870968  %
Enter 1 for loading 2 for training 3 for predicting1
Enter labelalphabet_F
31

Continue?0
Enter 1 for loading 2 for training 3 for predicting2
[21 20  7 ... 24 30  1] 98.76115625416278  %
Enter 1 for loading 2 for training 3 for predicting1
Enter labelalphabet_J
32

